In [1]:
import os
import io

import cv2 as cv
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from PIL import Image as pil_image
from IPython.html import widgets

from ipywidgets import interact, interact_manual
from IPython.display import display, Image, clear_output

/usr/local/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# Utilities

In [2]:
def imgs_show(imgs, rows, fontsize=36):
    fig = plt.figure()
    for n, (img, title) in enumerate(imgs):
        a = fig.add_subplot(rows, np.ceil(len(imgs)/float(rows)), n + 1)
        a.set_title(title, fontdict={"fontsize": fontsize})
        plt.imshow(img)
    fig.set_size_inches(np.array(fig.get_size_inches()) * len(imgs))
    plt.show()

def read_folder(folder, ext):
    images = []
    for filename in sorted(os.listdir(folder)):
        if filename.endswith(ext):
            path = os.path.join(folder, filename)
            images.append(np.array(pil_image.open(path)))
    return images

def img_show(img, title = '', figsize=(10, 5)):
    plt.figure(figsize=figsize)
    plt.gray()
    plt.imshow(img)
    plt.title(title)
    plt.show()

def display_frames_sequence(frames):
    def exec(i):
        a = np.uint8(frames[i])
        f = io.StringIO()

        imgByteArr = io.BytesIO()
        pil_image.fromarray(a, 'RGB').save(imgByteArr, format='PNG')
        display(Image(data=imgByteArr.getvalue()))
        
    widgets.interact(exec, i=widgets.IntSlider(min=0, max=len(frames) - 1, step=1, value=0))
    
def read_frames_dataset(path, ext='.jpg'):
    frames = read_folder(os.path.join(path, 'img/'), ext)
    df = pd.read_csv(os.path.join(path, 'groundtruth_rect.txt'), names=['x', 'y', 'w', 'h'])
    return frames, df

def enreach_single_frame_with_tracking_ractangle(frame, track_window, color=(0, 255, 0)):
    x,y,w,h = track_window
    return cv.rectangle(np.copy(frame), (x,y), (x+w,y+h), color, 2)

def enreach_frames_with_tracking_ractangle(frames, tracking_df, color=(0, 255, 0)):
    enreached_frames = []
    for i, frame in enumerate(frames):
        row = tracking_df.iloc[i]
        enreached_frame = enreach_single_frame_with_tracking_ractangle(
            frame, 
            (row['x'], row['y'], row['w'], row['h']),
            color
        )
        enreached_frames.append(enreached_frame)
    return enreached_frames